In [1]:
#pip install antropy

In [2]:
import parselmouth
import numpy as np
import librosa
import nolds
import pandas as pd
import joblib
import antropy as ant

In [3]:
def extract_features(audio_path):
    # Load audio
    sound = parselmouth.Sound(audio_path)
    y, sr = librosa.load(audio_path)

    # 1. MDVP:Fo(Hz), MDVP:Fhi(Hz), MDVP:Flo(Hz)
    pitch = sound.to_pitch()
    fo = pitch.selected_array['frequency']
    MDVP_Fo = np.mean(fo[fo > 0])  # Mean fundamental frequency
    MDVP_Fhi = np.max(fo)           # Max fundamental frequency
    MDVP_Flo = np.min(fo[fo > 0])   # Min fundamental frequency

    # 2. NHR and HNR
    harmonicity = sound.to_harmonicity()
    HNR = harmonicity.values[harmonicity.values != -200].mean()
    NHR = 1 / HNR if HNR != 0 else 0

    # 3. RPDE (Recurrence Period Density Entropy)
    RPDE = ant.perm_entropy(fo, normalize=True)    # or use a different signal for better accuracy

    # 4. DFA (Detrended Fluctuation Analysis)
    DFA = nolds.dfa(y)

    # 5. Spread2
    spread = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    Spread2 = spread.mean() / (sr / 2)

    # 6. D2 (Correlation Dimension)
    D2 = nolds.corr_dim(fo, emb_dim=25)  # Adjust emb_dim for accuracy, perhaps check paper

    # 7. PPE (Pitch Period Entropy)
    PPE = ant.app_entropy(fo)

    # Consolidate into dictionary
    features = {
        "MDVP:Fo(Hz)": MDVP_Fo,
        "MDVP:Fhi(Hz)": MDVP_Fhi,
        "MDVP:Flo(Hz)": MDVP_Flo,
        "NHR": NHR,
        "HNR": HNR,
        "RPDE": RPDE,
        "DFA": DFA,
        "Spread2": Spread2,
        "D2": D2,
        "PPE": PPE
    }
    
    return features

In [4]:
#audio_file = '../data/raw/HC_AH/AH_5.wav'
audio_file = '../data/raw/test.wav'
features = extract_features(audio_file)
print(features)

{'MDVP:Fo(Hz)': 128.20022909789762, 'MDVP:Fhi(Hz)': 198.78304100698804, 'MDVP:Flo(Hz)': 110.53636449125997, 'NHR': 0.06680996455476186, 'HNR': 14.967827129744006, 'RPDE': 0.7067980206076252, 'DFA': 0.5236775708533682, 'Spread2': 0.1956654099691866, 'D2': 0.777186218208257, 'PPE': 0.08720967544633418}


In [5]:
feature_df = pd.DataFrame(columns=[
    "MDVP:Fo(Hz)", "MDVP:Fhi(Hz)", "MDVP:Flo(Hz)", "NHR", "HNR", 
    "RPDE", "DFA", "Spread2", "D2", "PPE"
])

In [6]:
feature_df = pd.concat([feature_df, pd.DataFrame([features])], ignore_index=True)

C:\Users\bert_\AppData\Local\Temp\ipykernel_3356\1364591689.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  feature_df = pd.concat([feature_df, pd.DataFrame([features])], ignore_index=True)


In [7]:
feature_df.head()

,MDVP:Fo(Hz),MDVP:Fhi(Hz),MDVP:Flo(Hz),NHR,HNR,RPDE,DFA,Spread2,D2,PPE
0,128.200229,198.783041,110.536364,0.06681,14.967827,0.706798,0.523678,0.195665,0.777186,0.08721


In [8]:
test_model = joblib.load("../models/Custom_model.joblib")

In [9]:
prediction = test_model.predict(feature_df)

In [10]:
print("Predictions:", prediction)

Predictions: [1]


In [11]:
# Predict probabilities for each class
y_proba = test_model.predict_proba(feature_df)

# Example: Look at probabilities for the first few samples
print(y_proba[:5])

[[0.1238243 0.8761757]]
